In [47]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from time import sleep

## SIRV Epidemic Model w/ following epidemiliogical classes:
* S = susceptible
* I_s = infected w/ sensitive strain
* I_r = infected w/ resistant strain
* R = recovered
* V = vaccinated
* v = death rate = 1/v = lifespan
* y = recovery rate
* beta = transmission coef

--------------------------
* dS/dt = -B * I * S/N
* dI/dt = B * I * S/N - (y * I)
* dR/dt = y * I

In [83]:
def sim_SIR(N, I, days, y=0.2, beta=1.2):
    S = N
    I = I
    R = 0
    S_all = []
    I_all = []
    R_all = []

    # the model
    for i in range(0, days):
        dS = (- beta * I * (S/N))
        dI = (beta * I * (S/N) - (y * I))
        dR = (y * I)

        if (S+dS) < 0: S = 0      
        else: S += dS

        I += dI
        R += dR
        
        S_all.append(S)
        I_all.append(I)
        R_all.append(R)
    
    return S_all, I_all, R_all

In [89]:
def plot_SIR(N, I, days, y, beta):
    S_all, I_all, R_all = sim_SIR(N, I, days, y, beta)
    x = range(0, days)
    plt.figure(figsize=[14,4])
    plt.fill_between(x, S_all, color='tab:green', alpha=0.25, label='Susceptible')
    plt.fill_between(x, I_all, color='tab:orange', alpha=0.25, label='Infected')
    plt.fill_between(x, R_all, color='tab:blue', alpha=0.25, label='Recovered')
    plt.plot(S_all, color='tab:green', lw=2)
    plt.plot(I_all, color='tab:orange', lw=2)
    plt.plot(R_all, color='tab:blue', lw=2)
    plt.xlabel('Days')
    plt.ylabel('Population')

interact(plot_SIR,
         N=widgets.IntSlider(min=100, max=10000, step=100, value=1000),
         I=widgets.IntSlider(min=1, max=100, step=1, value=1),
         days=widgets.IntSlider(min=10, step=1, value=50),
         y=widgets.FloatSlider(min=0.0, max=1.0, step=0.02, value=0.2),
         beta=widgets.FloatSlider(min=1.0, step=0.1, value=1.2))

interactive(children=(IntSlider(value=1000, description='N', max=10000, step=100), IntSlider(value=1, descript…

<function __main__.plot_SIR(N, I, days, y, beta)>